In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn import linear_model

# data gathering & cleaning

In [2]:
nasdaq_100_tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "GOOG", "TSLA", "META", "NVDA", "PYPL", "NFLX",
    "ASML", "ADBE", "INTC", "CMCSA", "CSCO", "PEP", "AVGO", "TMUS", "COST", "PDD",
    "TXN", "QCOM", "AMAT", "MU", "AMGN", "INTU", "ISRG", "ZM", "CSX", "VRTX",
    "JD", "GILD", "BIDU", "MRVL", "REGN", "MDLZ", "ADSK", "ATVI", "BIIB", "ILMN",
    "LRCX", "ADP", "BKNG", "MELI", "KLAC", "DOCU", "NXPI", "MNST", "WDAY", "ROST",
    "KDP", "EA", "ALGN", "ADI", "IDXX", "DXCM", "XEL", "CTAS", "EXC", "MAR",
    "SNPS", "ASAN", "CDNS", "CPRT", "SGEN", "SPLK", "ORLY", "DLTR", "MTCH",
    "MCHP", "INCY", "PCAR", "CTSH", "FAST", "VRSK", "CHKP", "FOXA", "FOX", "ANSS",
    "SWKS", "OKTA", "TTD", "CDW", "TEAM", "WBA", "LULU", "PAYX",
    "VRSN", "AEP", "ZBRA", "PTON", "TCOM", "NTES", "BMRN", "ULTA", "EXPE",
    "CSGP", "SIRI", "EBAY", "WDC"
    ]
end_date = '2020-12-31'
start_date = '2016-3-28'
data = pd.DataFrame()
daily_log_ret = pd.DataFrame()
for ticker in nasdaq_100_tickers:
    stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    data[ticker] = stock_data['Adj Close']
    data[ticker].index = data[ticker].index.strftime('%Y/%m/%d')
    daily_log_return = np.log(data[ticker] / data[ticker].shift(1))
    daily_log_ret[ticker] = daily_log_return


In [3]:
daily_log_ret=daily_log_ret.iloc[1: , :]

In [4]:
zeros=daily_log_ret.isnull().sum().to_frame()

In [5]:
zeros[zeros.iloc[:, 0]!=0]

,0
PDD,588
ZM,771
DOCU,526
ASAN,1137
FOXA,744
FOX,745
OKTA,261
TTD,124
PTON,882


In [6]:
drop_lst=['PDD', 'ZM', 'DOCU', 'ASAN', 'FOXA', 'FOX', 'OKTA', 'TTD', 'PTON']

In [7]:
daily_log_ret=daily_log_ret.drop(columns=drop_lst)

In [9]:
alpha=pd.read_csv('first round alpha.csv', header=0)
alpha=alpha.set_index(['ticker', 'Date'])

In [10]:
alpha.replace([np.inf, -np.inf], np.nan, inplace=True)


In [11]:
alpha=alpha.ffill()

alpha_nor = alpha.groupby('ticker').transform(lambda x: (x-x.mean())/x.std(ddof=1))
alpha_nor

In [12]:
from sklearn.preprocessing import scale
alpha_nor=alpha.groupby('ticker').transform(lambda x: scale(x))
alpha_nor

a49       a51       a12      a101       a10       a18  \
ticker Date                                                                     
AAPL   2016/03/29 -0.400829 -0.400829 -0.532299  1.684680  0.299261  1.329465   
       2016/03/30 -0.286141 -0.286141 -0.417634  0.806602 -1.237635 -1.013106   
       2016/03/31  0.174490  0.174490 -0.171388 -1.513888  0.518817  0.743823   
       2016/04/01 -0.120690 -0.120690  0.123733  1.135641 -0.084963  0.626694   
       2016/04/04 -0.145133 -0.145133 -0.276655  0.547944 -1.731637 -1.833006   
...                     ...       ...       ...       ...       ...       ...   
ZM     2020/12/23  3.431254  3.431989  2.649921 -2.828854  2.451380  3.107069   
       2020/12/24  1.265494  1.266024 -1.482855 -3.165956  2.451380  3.026021   
       2020/12/28  3.254404  3.255122  2.486223 -2.906328  2.451380  3.107069   
       2020/12/29 -0.195499 -0.195108 -0.130512  0.008895 -0.468807 -0.702196   
       2020/12/30  0.162160  0.162586 -0.461573 -1.829547  1.532670  0.067762   

                        a17       a46       a38       a34  ...       a83  \
ticker Date                                                ...             
AAPL   2016/03/29 -0.351527  0.028189  0.613764 -0.719891  ... -0.442461   
       2016/03/30 -2.105818  0.558937 -1.376985  0.371947  ... -0.999560   
       2016/03/31  0.790978 -1.047365  0.347425 -0.236963  ...  1.383585   
       2016/04/01 -0.088845 -0.495528  0.035394  0.413941  ... -1.030510   
       2016/04/04 -3.117089  0.558937 -1.787338  0.015000  ... -1.123360   
...                     ...       ...       ...       ...  ...       ...   
ZM     2020/12/23  1.198582 -0.172244  1.825700  1.526224  ...  0.224218   
       2020/12/24  1.057193 -0.172244  1.912998  2.054910  ...  1.466574   
       2020/12/28  1.189100 -0.172244  1.909043  2.242508  ...  0.289606   
       2020/12/29  0.500622 -0.172244  1.655060 -0.537357  ... -2.718203   
       2020/12/30  1.140493  1.187231  1.859602  2.378943  ...  1.924284   

                        a47        a8       a16       a30       a45        a7  \
ticker Date                                                                     
AAPL   2016/03/29 -0.483120 -0.366619  0.239361  0.045363 -0.911512  0.090689   
       2016/03/30 -1.160297 -0.548389  0.174323  0.047793 -0.736041  0.090689   
       2016/03/31 -0.043848  0.663416 -1.419112  0.052531 -0.557587  0.090689   
       2016/04/01 -0.435273  0.239284 -0.963845  0.059206  0.870314  0.090689   
       2016/04/04 -0.561302 -0.245438  1.637682  0.057928 -0.569792  0.090689   
...                     ...       ...       ...       ...       ...       ...   
ZM     2020/12/23  0.765861 -2.370115 -0.358556 -0.001390  0.278551  1.123306   
       2020/12/24  0.286852  0.953482 -0.862684  0.137678 -2.659117  0.081949   
       2020/12/28  1.201424  1.108068  0.649698  0.228420  0.308426  3.206019   
       2020/12/29  0.648573  1.108068  0.541671  0.152370  0.388091  0.081949   
       2020/12/30  0.689495  1.108068 -2.086992  0.425707 -2.698950  0.081949   

                        a14       a55       a60  
ticker Date                                      
AAPL   2016/03/29  0.508636  0.092721 -0.770235  
       2016/03/30 -1.264395  0.092721 -0.709932  
       2016/03/31 -0.332069 -1.328711  1.135329  
       2016/04/01 -0.185518 -1.276919 -0.830538  
       2016/04/04 -0.851576 -1.170985  1.358449  
...                     ...       ...       ...  
ZM     2020/12/23 -0.968517  0.608767  2.215013  
       2020/12/24 -0.962129  1.221916  2.393061  
       2020/12/28 -0.931973  1.588557  1.989485  
       2020/12/29 -0.571372  1.423771  1.301031  
       2020/12/30 -0.821938  1.946106 -2.129367  

[120000 rows x 27 columns]

# Lasso

In [13]:
daily_log_ret.columns

Index(['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'GOOG', 'TSLA', 'META', 'NVDA', 'PYPL',
       'NFLX', 'ASML', 'ADBE', 'INTC', 'CMCSA', 'CSCO', 'PEP', 'AVGO', 'TMUS',
       'COST', 'TXN', 'QCOM', 'AMAT', 'MU', 'AMGN', 'INTU', 'ISRG', 'CSX',
       'VRTX', 'JD', 'GILD', 'BIDU', 'MRVL', 'REGN', 'MDLZ', 'ADSK', 'ATVI',
       'BIIB', 'ILMN', 'LRCX', 'ADP', 'BKNG', 'MELI', 'KLAC', 'NXPI', 'MNST',
       'WDAY', 'ROST', 'KDP', 'EA', 'ALGN', 'ADI', 'IDXX', 'DXCM', 'XEL',
       'CTAS', 'EXC', 'MAR', 'SNPS', 'CDNS', 'CPRT', 'SGEN', 'SPLK', 'ORLY',
       'DLTR', 'MTCH', 'MCHP', 'INCY', 'PCAR', 'CTSH', 'FAST', 'VRSK', 'CHKP',
       'ANSS', 'SWKS', 'CDW', 'TEAM', 'WBA', 'LULU', 'PAYX', 'VRSN', 'AEP',
       'ZBRA', 'TCOM', 'NTES', 'BMRN', 'ULTA', 'EXPE', 'CSGP', 'SIRI', 'EBAY',
       'WDC'],
      dtype='object')

In [50]:
clf = linear_model.Lasso(alpha=0.001)

In [51]:
clf.fit(x, y)

Lasso(alpha=0.001)

In [53]:
clf.coef_

array([-0.02823392, -0.        , -0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        ,  0.        ,
       -0.        ,  0.        , -0.        , -0.        ,  0.        ,
        0.        ,  0.        , -0.00024351, -0.        ,  0.        ,
       -0.        ,  0.        ,  0.        , -0.        ,  0.        ,
       -0.        , -0.        ])

In [54]:
y_pred=clf.predict(x)

In [55]:
clf.score(x, y)

0.9055546892965576

In [14]:
penalty=np.logspace(np.log10(0.0007), np.log10(0.001), 100)
penalty

array([0.0007    , 0.00070253, 0.00070506, 0.00070761, 0.00071016,
       0.00071272, 0.0007153 , 0.00071788, 0.00072047, 0.00072307,
       0.00072568, 0.0007283 , 0.00073093, 0.00073357, 0.00073621,
       0.00073887, 0.00074154, 0.00074421, 0.0007469 , 0.0007496 ,
       0.0007523 , 0.00075502, 0.00075774, 0.00076048, 0.00076322,
       0.00076598, 0.00076874, 0.00077151, 0.0007743 , 0.00077709,
       0.0007799 , 0.00078271, 0.00078554, 0.00078837, 0.00079122,
       0.00079407, 0.00079694, 0.00079982, 0.0008027 , 0.0008056 ,
       0.00080851, 0.00081143, 0.00081436, 0.00081729, 0.00082024,
       0.00082321, 0.00082618, 0.00082916, 0.00083215, 0.00083515,
       0.00083817, 0.00084119, 0.00084423, 0.00084728, 0.00085033,
       0.0008534 , 0.00085648, 0.00085958, 0.00086268, 0.00086579,
       0.00086892, 0.00087205, 0.0008752 , 0.00087836, 0.00088153,
       0.00088471, 0.0008879 , 0.00089111, 0.00089432, 0.00089755,
       0.00090079, 0.00090404, 0.00090731, 0.00091058, 0.00091

In [15]:
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "GOOG", "TSLA", "META", "NVDA", "PYPL", "NFLX",
    "ASML", "ADBE", "INTC", "CMCSA", "CSCO", "PEP", "AVGO", "TMUS", "COST",
    "TXN", "QCOM", "AMAT", "MU", "AMGN", "INTU", "ISRG", "CSX", "VRTX",
    "JD", "GILD", "BIDU", "MRVL", "REGN", "MDLZ", "ADSK", "ATVI", "BIIB", "ILMN",
    "LRCX", "ADP", "BKNG", "MELI", "KLAC", "NXPI", "MNST", "WDAY", "ROST",
    "KDP", "EA", "ALGN", "ADI", "IDXX", "DXCM", "XEL", "CTAS", "EXC", "MAR",
    "SNPS", "CDNS", "CPRT", "SGEN", "SPLK", "ORLY", "DLTR", "MTCH",
    "MCHP", "INCY", "PCAR", "CTSH", "FAST", "VRSK", "CHKP", "ANSS",
    "SWKS", "CDW", "TEAM", "WBA", "LULU", "PAYX",
    "VRSN", "AEP", "ZBRA", "TCOM", "NTES", "BMRN", "ULTA", "EXPE",
    "CSGP", "SIRI", "EBAY", "WDC"
    ]

for ticker in tickers:
    n=(result.loc[ticker].abs()>0.00000001).value_counts().loc[True]
    print(ticker,n)

In [16]:
alpha_test=pd.read_csv('backtesting alpha.csv', header=0)
alpha_test=alpha_test.set_index(['ticker', 'Date'])
alpha_test

a51       a49      a101       a12       a10  \
ticker Date                                                           
AAPL   2021/02/01 -0.350905 -0.350905  0.094158  0.652992  0.391199   
       2021/02/02 -0.350905 -0.350905 -0.848727  0.186552  0.758392   
       2021/02/03  0.363932  0.363932 -1.583264  0.256693 -0.080906   
       2021/02/04 -1.205221 -1.205221  0.635622  1.098395 -1.077572   
       2021/02/05  0.217479  0.217479 -0.745784 -0.332502  0.810848   
...                     ...       ...       ...       ...       ...   
ZBRA   2022/12/23 -0.105258 -0.105258  0.574980  0.038514 -0.779260   
       2022/12/27 -0.320287 -0.320287  0.458986 -0.323464 -1.321158   
       2022/12/28  0.400635  0.400635 -0.829647 -0.466596  0.846437   
       2022/12/29 -1.141971 -1.141971  1.601207 -1.143878 -1.200736   
       2022/12/30  0.084841  0.084841  0.808022 -0.151291  0.695910   

                            a28           a17       a42       a38       a83  \
ticker Date                                                                   
AAPL   2021/02/01 -2.951353e-17 -5.856407e-16  0.689784  0.742978 -1.150895   
       2021/02/02 -2.951353e-17 -5.856407e-16  2.295837  1.076694  1.554392   
       2021/02/03 -2.951353e-17 -5.856407e-16  0.556083  0.895690  1.335044   
       2021/02/04 -2.951353e-17 -5.856407e-16 -0.471284  0.323171  0.274864   
       2021/02/05 -2.951353e-17 -5.856407e-16  0.176340  0.808383  1.444718   
...                         ...           ...       ...       ...       ...   
ZBRA   2022/12/23 -2.445667e-01 -4.517829e-01 -0.433624 -0.304386  1.013050   
       2022/12/27 -2.682931e-01 -1.469818e+00 -1.064582 -2.419566 -0.559839   
       2022/12/28  8.191404e-01  6.332360e-01  1.293208  0.059146 -1.410049   
       2022/12/29 -1.421697e+00 -4.078221e+00 -1.164207 -3.190638  1.268113   
       2022/12/30 -8.404127e-01  5.197087e-01 -0.832124 -1.381346  1.353134   

                   ...           a47       a60        a5           a45  \
ticker Date        ...                                                   
AAPL   2021/02/01  ...  2.520799e-16 -0.681790 -0.418277 -7.683592e-17   
       2021/02/02  ...  1.200280e+00  1.165857 -1.446931 -7.683592e-17   
       2021/02/03  ...  1.030061e+00  1.108298 -1.325775 -7.683592e-17   
       2021/02/04  ... -4.876862e-02 -1.067436 -0.709833 -7.683592e-17   
       2021/02/05  ... -4.842722e-02 -1.188310 -1.110820 -7.683592e-17   
...                ...           ...       ...       ...           ...   
ZBRA   2022/12/23  ... -4.449050e-01  1.004733 -0.091802  1.237499e+00   
       2022/12/27  ... -8.429269e-01 -0.477891  0.450819 -2.267877e-01   
       2022/12/28  ... -6.695655e-01 -1.543829 -0.488378 -1.398217e+00   
       2022/12/29  ... -1.626787e+00  1.372967  1.910078 -1.343065e-01   
       2022/12/30  ... -1.476254e+00  1.886555  1.767595 -6.737804e-01   

                            a30       a13        a8       a11       a16  \
ticker Date                                                               
AAPL   2021/02/01 -2.200849e-17  1.247913  1.367771 -0.822079  1.223905   
       2021/02/02 -1.824217e-01 -0.548785  1.470114 -0.914457 -0.744590   
       2021/02/03 -3.484907e-01 -1.566913  1.521285 -0.919100 -1.919810   
       2021/02/04 -1.120961e-02 -1.566913  1.367771 -0.897839 -1.479102   
       2021/02/05  1.431212e-01 -1.387244  1.674798 -0.804239 -1.067775   
...                         ...       ...       ...       ...       ...   
ZBRA   2022/12/23 -1.159585e-02 -1.594089 -1.045035 -0.843067  0.334676   
       2022/12/27 -3.139533e-02  0.158878 -1.130043 -1.318860  0.306303   
       2022/12/28 -1.423671e-02  0.509471  0.598451  1.248713  0.618409   
       2022/12/29 -4.953364e-02  0.976929  0.258419 -0.479043  1.356115   
       2022/12/30 -1.918415e-01  1.093793 -0.818347 -0.201950  0.845396   

                        a55  
ticker Date                  
AAPL   2021/02/01  1.062330  
       2021/02/02 -0.196074  
       2021/0

In [17]:
alpha_test=alpha_test[alpha_nor.columns]

In [26]:
x=alpha_nor.loc[['TSLA']]
x_test = alpha_test.loc[['TSLA']]
y=daily_log_ret['TSLA']
clf=linear_model.Lasso(alpha=0.006)
clf.fit(x, y)
y_predict = clf.predict(x_test)
y_predict

array([ 0.01501372,  0.01846341, -0.0227553 , -0.00314688,  0.00970809,
        0.00430724, -0.01092323, -0.03742483,  0.00580413,  0.0134384 ,
       -0.02264359,  0.01322277,  0.00302649, -0.0142951 , -0.04777934,
        0.00153615,  0.03691261, -0.04116061, -0.01139791,  0.0404012 ,
       -0.03062237, -0.02728804, -0.02352157, -0.0181858 , -0.02741139,
        0.0757441 , -0.01010035,  0.02032693,  0.015934  ,  0.02000987,
       -0.02657497,  0.03561566, -0.03816411,  0.00869885,  0.00131498,
       -0.01422863, -0.03278402,  0.02312964, -0.01916153, -0.00533099,
        0.03024598,  0.03225865, -0.01580707,  0.00966682,  0.00479288,
       -0.02058664,  0.01627628, -0.00221312,  0.03458834,  0.05837741,
       -0.02750838,  0.00269789,  0.01205523, -0.01388832,  0.00769729,
        0.0327515 , -0.02272747,  0.01499995,  0.00440236, -0.02755686,
       -0.00539492, -0.02002459,  0.03752485, -0.02454453, -0.00742714,
       -0.00952425, -0.01217308,  0.01155168, -0.03841324,  0.00

In [34]:
pd.DataFrame(columns=['tsla'], index=x_test.index)

tsla
ticker Date           
TSLA   2021/02/01  NaN
       2021/02/02  NaN
       2021/02/03  NaN
       2021/02/04  NaN
       2021/02/05  NaN
...                ...
       2022/12/23  NaN
       2022/12/27  NaN
       2022/12/28  NaN
       2022/12/29  NaN
       2022/12/30  NaN

[484 rows x 1 columns]

In [37]:
pd.DataFrame(columns=tickers, index=alpha_test.loc['AAPL'].index)

,AAPL,MSFT,AMZN,GOOGL,GOOG,TSLA,META,NVDA,PYPL,NFLX,...,ZBRA,TCOM,NTES,BMRN,ULTA,EXPE,CSGP,SIRI,EBAY,WDC
Date,,,,,,,,,,,,,,,,,,,,,
2021/02/01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021/02/02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021/02/03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021/02/04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021/02/05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022/12/23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022/12/27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022/12/28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
alpha5_result=pd.DataFrame(columns=alpha.columns)
alpha5_penalty_result=pd.DataFrame(columns=['penalty'])
backtest_y=pd.DataFrame(columns=tickers, index=alpha_test.loc['AAPL'].index)
penalty=np.logspace(np.log10(0.00001), np.log10(0.006), 200)
for ticker in tickers:
    for i in penalty:
        x=alpha_nor.loc[[ticker]]
        x_test = alpha_test.loc[[ticker]]
        y=daily_log_ret[ticker]
        clf=linear_model.Lasso(alpha=i)
        clf.fit(x, y)
        n=np.count_nonzero(clf.coef_)
        if n<=6 and n>=4:
            alpha5_result.loc[ticker]=clf.coef_
            alpha5_penalty_result.loc[ticker] = i
            y_predict = clf.predict(x_test)
            backtest_y[ticker]=y_predict
            break
        

/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.609e-02, tolerance: 1.599e-04
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.429e-02, tolerance: 1.599e-04
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.378e-05, tolerance: 3.236e-05
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.374e-05, tolerance: 3.236e-05
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.143e-05, tolerance: 3.236e-05
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.132e-05, tolerance: 3.236e-05
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.448e-05, tolerance: 3.236e-05
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.413e-05, tolerance: 3.236e-05
  model = cd_fast.enet_coordinate_descent(
/Users/zhangxinyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

In [42]:
alpha5_result

,a49,a51,a12,a101,a10,a18,a17,a46,a38,a34,...,a83,a47,a8,a16,a30,a45,a7,a14,a55,a60
AAPL,-0.014577,-0.000000e+00,-0.000000,0.002438,-0.000000,0.0,-0.000006,0.000324,-0.0,0.0,...,-0.00000,-0.000000e+00,-0.000000,-0.0,-0.0,-0.0,-0.000471,-0.000000,-0.0,-0.000000
MSFT,-0.014117,-2.035409e-17,-0.000000,0.001838,-0.000222,-0.0,-0.000021,0.000000,-0.0,-0.0,...,-0.00000,-0.000000e+00,-0.000000,-0.0,-0.0,-0.0,-0.000000,-0.000000,-0.0,-0.000000
AMZN,-0.014309,-1.073216e-17,-0.000000,0.002431,-0.000000,0.0,-0.000000,0.000000,-0.0,-0.0,...,-0.00000,-1.020118e-03,0.000000,-0.0,0.0,-0.0,-0.000000,-0.000000,-0.0,-0.000000
GOOGL,-0.015554,-0.000000e+00,0.000000,0.000782,-0.000000,0.0,-0.000000,0.000000,-0.0,-0.0,...,-0.00000,-3.299064e-05,0.000000,-0.0,-0.0,-0.0,-0.000109,-0.000000,-0.0,-0.000013
GOOG,-0.015431,-0.000000e+00,-0.000000,0.000851,-0.000000,0.0,-0.000000,0.000059,-0.0,-0.0,...,-0.00000,-4.762101e-07,0.000000,-0.0,-0.0,-0.0,-0.000063,-0.000000,-0.0,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EXPE,-0.026494,-0.000000e+00,0.001700,-0.000000,0.000594,0.0,0.000000,-0.000000,0.0,0.0,...,-0.00000,0.000000e+00,0.000018,-0.0,0.0,-0.0,-0.000000,-0.000032,0.0,0.000000
CSGP,-0.014618,-0.000000e+00,-0.000000,0.003349,0.000000,0.0,-0.000000,0.000443,-0.0,0.0,...,0.00000,-3.600891e-04,0.000000,-0.0,-0.0,-0.0,-0.000464,-0.000000,-0.0,0.000000
SIRI,-0.017298,-0.000000e+00,0.000000,0.000054,0.000030,-0.0,-0.000000,-0.000000,-0.0,0.0,...,-0.00000,0.000000e+00,-0.000000,-0.0,-0.0,-0.0,0.000000,-0.000044,-0.0,-0.000091
EBAY,-0.016648,-0.000000e+00,-0.000004,0.000406,0.000000,0.0,-0.000000,0.000437,-0.0,0.0,...,-0.00000,-0.000000e+00,-0.000000,-0.0,-0.0,-0.0,-0.000122,-0.000000,-0.0,-0.000005


In [43]:
backtest_y

,AAPL,MSFT,AMZN,GOOGL,GOOG,TSLA,META,NVDA,PYPL,NFLX,...,ZBRA,TCOM,NTES,BMRN,ULTA,EXPE,CSGP,SIRI,EBAY,WDC
Date,,,,,,,,,,,,,,,,,,,,,
2021/02/01,0.007151,0.005679,0.011462,0.008385,0.008532,0.015463,0.004485,0.009046,0.008586,0.004410,...,0.006430,0.025616,0.010881,0.013394,0.002307,0.005654,0.013881,0.150687,0.015598,0.019812
2021/02/02,0.004438,0.001893,0.007927,0.007527,0.007538,0.018984,0.005087,0.013347,0.010101,0.005657,...,0.008053,0.024719,0.012334,0.013399,0.002332,0.009703,0.006771,0.150991,0.012189,0.018861
2021/02/03,-0.007726,0.013241,-0.015315,0.046832,0.047457,-0.023622,0.000918,0.000225,0.012279,-0.013475,...,-0.001589,-0.004259,-0.000721,-0.006716,0.004970,0.015371,0.000188,-0.003111,0.010247,-0.003701
2021/02/04,0.020549,-0.002970,0.006078,-0.001235,-0.002026,-0.003016,0.000435,0.014045,0.065844,0.023865,...,0.013500,0.022516,-0.000562,0.029911,0.020078,0.024226,0.000825,-0.009106,0.040931,0.029732
2021/02/05,-0.003535,0.001441,0.008459,0.013364,0.013481,0.010285,0.006594,-0.004543,0.001685,-0.000128,...,0.001541,0.050635,0.015635,0.005338,0.019491,-0.002313,-0.012494,-0.001540,0.016031,0.010392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022/12/23,0.001363,0.004345,0.010827,0.011860,0.012462,-0.014187,0.005338,-0.005773,0.009191,-0.002930,...,0.006873,-0.019273,-0.007746,-0.020668,-0.002767,0.012485,-0.002096,0.004440,0.001497,0.000317
2022/12/27,-0.009458,-0.005859,-0.011805,-0.012256,-0.012521,-0.044060,-0.002055,-0.044436,-0.001059,-0.016778,...,0.012877,0.032981,0.019446,-0.033716,0.022956,0.001002,-0.000131,-0.009255,-0.000465,0.011651
2022/12/28,-0.022254,-0.007528,-0.005860,-0.008760,-0.009494,0.015494,-0.001868,0.000107,-0.000946,-0.011142,...,-0.007133,-0.023920,-0.010414,-0.006857,-0.002557,-0.015300,-0.007500,-0.007759,-0.001457,-0.017293


In [46]:
backtest_y.isna().sum()

AAPL     0
MSFT     0
AMZN     0
GOOGL    0
GOOG     0
        ..
EXPE     0
CSGP     0
SIRI     0
EBAY     0
WDC      0
Length: 91, dtype: int64

In [174]:
alpha5_result.to_csv('lasso_alpha.csv')

In [47]:
backtest_y.to_csv('backtest y.csv')